In [39]:
import os
import sys
import jinja2
import pandas as pd
import xml.etree.ElementTree as ET
from six.moves.urllib.request import urlretrieve

In [40]:
last_percent_reported = None
url = 'https://www.comprasestatales.gub.uy/comprasenlinea/jboss/generarReporte?tipo_publicacion=lv&anio_inicial=2017&mes_inicial=07&dia_inicial=01&hora_inicial=0&anio_final=2017&mes_final=07&dia_final=10&hora_final=24'

In [62]:
def download_progress_hook(count, blockSize, totalSize):
    """ 
        A hook to report the progress of a download. This is mostly intended 
        for users with slow internet connections. Reports every 5% change in download progress.
    """
    global last_percent_reported
    percent = (count + 1) * blockSize

    if last_percent_reported != percent:
        if percent % 5 == 0:
            sys.stdout.write("%s%%" % percent)
            sys.stdout.flush()
        else:
            sys.stdout.write(".")
            sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(url, filename, directory, force=False):
    """Download a file if not present, and make sure it's the right size."""
    dest_filename = os.path.join(directory, filename)
    os.makedirs(name=directory, exist_ok=True) # Attempt to create the dir where the file should be downloaded to. If exists then do nothing
    if force or not os.path.exists(dest_filename):
        print('Iniciando Descarga:', url) 
        try:
            filename, _ = urlretrieve(url, dest_filename, reporthook=download_progress_hook)
            print('Descarga Completa!')
        except:
            print('%s no se pudo descargar!' % filename)
    else:
        print('%s ya existe en la carpeta - Salteando la descarga.' % dest_filename) 
    
    return dest_filename

def get_contacts(filename):
    """
    Return two lists names, emails containing names and email addresses
    read from a file specified by filename.
    """
    
    names = []
    emails = []
    with open(filename, mode='r', encoding='utf-8') as contacts_file:
        for a_contact in [c for c in contacts_file if not c.startswith("#")]:
            names.append(a_contact.split()[0])
            emails.append(a_contact.split()[1])
    return names, emails

def xml2df(xml_file):
    #xml_data = open(xml_file).read()

    root = ET.parse(xml_file).getroot().getchildren()[1]#ET.XML(xml_data) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        print(i)
        print(child)
        for subchild in child:
            record[subchild.tag] = subchild.text
            all_records.append(record)
            print(subchild.tag)
    return pd.DataFrame(all_records)

In [42]:
maybe_download(url, 'compras.xml', 'data', force=True)

Iniciando Descarga: https://www.comprasestatales.gub.uy/comprasenlinea/jboss/generarReporte?tipo_publicacion=lv&anio_inicial=2017&mes_inicial=07&dia_inicial=01&hora_inicial=0&anio_final=2017&mes_final=07&dia_final=10&hora_final=24
..Descarga Completa!


'data/compras.xml'

In [64]:
e = xml.etree.ElementTree.parse('data/compras.xml').getroot().getchildren()[1].getchildren()

In [67]:
e = e[0]

In [74]:
e.attrib

{'anio_compra': '2017',
 'email_contacto': 'uteabast@ute.com.uy ',
 'estado_compra': '4',
 'fax_contacto': '(0598) 22083428     ',
 'fecha_hora_apertura': '12/09/2017 10:00',
 'fecha_hora_tope_entrega': '12/09/2017 10:00',
 'fecha_publicacion': '11/05/2017 12:30',
 'fecha_sol_aclar': '07/09/2017',
 'fecha_sol_prorr': '07/09/2017',
 'fecha_ult_mod_llamado': '10/07/2017 18:30',
 'fondos_rotatorios': 'N',
 'id_compra': 'i247250',
 'id_inciso': '61',
 'id_tipocompra': 'LA',
 'id_ue': '1',
 'lugar_apertura': 'SALA DE APERTURAS :UTE Palacio de la Luz,Paraguay 2431 Planta Baja,Montevideo - Uruguay.',
 'lugar_compra_pliego': '',
 'lugar_entrega': 'Depto. Atención Personalizada a Proveedores,UTE Palacio de la Luz,Paraguay 2431 Planta Baja.Montevideo - Uruguay.  ',
 'nombre_contacto': 'Depto. Atención Personalizada a Proveedores                                                        ',
 'nombre_pliego': 'pliego_i247250.pdf',
 'nro_ampliacion': '0',
 'num_compra': 'Y50355',
 'objeto': 'Dispositiv

In [ ]:
reporte_dato = e.getchildren()[1]

In [ ]:
compras = reporte_dato.getchildren()

In [ ]:
compra = compras[0]

In [49]:
import xmltodict

In [50]:
import codecs
with codecs.open('data/compras.xml',"r",encoding='utf-8', errors='ignore') as fd:
    doc = xmltodict.parse(fd.read())

In [53]:
doc

OrderedDict([('reporte',
              OrderedDict([('reporte_cabezal',
                            OrderedDict([('@fecha_generado',
                                          '03/08/2017 07:38'),
                                         ('@cant_compras', '1')])),
                           ('reporte_dato',
                            OrderedDict([('compra',
                                          OrderedDict([('@id_compra',
                                                        'i247250'),
                                                       ('@id_inciso', '61'),
                                                       ('@id_ue', '1'),
                                                       ('@num_compra',
                                                        'Y50355'),
                                                       ('@anio_compra',
                                                        '2017'),
                                                       ('@nro_ampliacion',
   

In [ ]:
dc = doc['reporte']['reporte_dato']['compra']

In [ ]:
comps = [(c.get('id_compra'), c.get('id_inciso'), c.get('nombre_pliego'),
          c.get('fecha_publicacion'), c.get('fecha_ult_mod_llamado'), c.get('fecha_hora_apertura')) 
         for c in dc]

In [10]:
import pandas as pd
#compras = pd.DataFrame(comps, columns=['id_compra', 'id_inciso', 'nombre_pliego', 'fecha_publicacion', 'fecha_ult_mod_llamado', 'fecha_hora_apertura'])
compras = pd.DataFrame()

In [ ]:
compras = compras.from_dict(data=dc)

In [ ]:
import jinja2

In [ ]:
loader = jinja2.FileSystemLoader('./templates')
jenv = jinja2.Environment(loader=loader)

In [ ]:
email_template = jenv.get_template('email.html')

In [ ]:
email = email_template.render(saludo='Estimado', usuario='Matias',compras=compras)

In [ ]:
with open('notif.html', 'w') as n:
    n.write(email)

In [ ]:
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_mail(me, you, subject, html, text):
    # me == my email address
    # you == recipient's email address

    # Create message container - the correct MIME type is multipart/alternative.
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = me
    msg['To'] = you

    # Record the MIME types of both parts - text/plain and text/html.
    part1 = MIMEText(text, 'plain')
    part2 = MIMEText(html, 'html')

    # Attach parts into message container.
    # According to RFC 2046, the last part of a multipart message, in this case
    # the HTML message, is best and preferred.
    msg.attach(part1)
    msg.attach(part2)

    # Send the message via local SMTP server.
    s = smtplib.SMTP(host='smtp-mail.outlook.com', port=587)
    
    s.starttls()
    s.login('daniel.bonini@outlook.com', 'C0ntra$3nha001')
    # sendmail function takes 3 arguments: sender's address, recipient's address
    # and message to send - here it is sent as one string.
    s.sendmail(me, you, msg.as_string())
    del msg
    s.quit()

In [ ]:
send_mail('daniel.bonini@outlook.com', 'm.martinezlaurini@gmail.com', 'Mail de Prueba', email, 'Este es un mail de prueba')

In [1]:
from string import Template

In [3]:
url = Template('https://www.comprasestatales.gub.uy/comprasenlinea/jboss/generarReporte?tipo_publicacion=$TP&anio_inicial=$AI&mes_inicial=$MI&dia_inicial=$DI&hora_inicial=$HI&anio_final=$AF&mes_final=$MF&dia_final=$DF&hora_final=$HF')

In [5]:
url.substitute(TP='lv',AI=2017,MI=7,DI=11,HI=0,AF=2017,MF=7,DF=21,HF=24)

'https://www.comprasestatales.gub.uy/comprasenlinea/jboss/generarReporte?tipo_publicacion=lv&anio_inicial=2017&mes_inicial=7&dia_inicial=11&hora_inicial=0&anio_final=2017&mes_final=7&dia_final=21&hora_final=24'

In [6]:
lista = ["#hola", "hola#", "hola"]


In [9]:
[s for s in lista if not s.startswith("#")]

['hola#', 'hola']

In [6]:
nombres, _ = get_contacts('contactos.txt')
print( nombres)

['daniel']
